In [ ]:
%pip install langchain
%pip install langchain-deepseek
%pip install langchain-ollama
%pip install python-dotenv

In [ ]:

from dotenv import load_dotenv
import os

# 自动从 .env 加载变量到系统环境
load_dotenv(override=True)  

os.environ

# 读取变量
# DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
# print(DEEPSEEK_API_KEY)

In [ ]:
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_retries=2,
    # other params...
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="deepseek-r1:latest",
    temperature=0,
    max_retries=2,
    # other params...
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg)

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

os.environ["DEEPSEEK_API_KEY"] = os.environ["SF_API_KEY"]
os.environ["DEEPSEEK_API_BASE"] = os.environ["SF_API_BASE"]

# print(os.environ["SF_API_BASE"])
# print(os.environ["SF_API_KEY"])

llm = ChatDeepSeek(
    model="deepseek-ai/DeepSeek-V3",
    temperature=0,
    max_retries=2,
    # other params...
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to chinese. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

In [ ]:
from langchain_core.runnables import RunnableBranch
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

import os
from dotenv import load_dotenv

load_dotenv(override=True)

os.environ["DEEPSEEK_API_KEY"] = os.environ["SF_API_KEY"]
os.environ["DEEPSEEK_API_BASE"] = os.environ["SF_API_BASE"]

llm = ChatDeepSeek(
    model="deepseek-ai/DeepSeek-V3",
    temperature=0,
    max_retries=2,
    # other params...
)
input_data = "What is the capital of France?"
summary_chain = (
    ChatPromptTemplate.from_template("Summarize this: {input}")
    | llm
    | StrOutputParser()
)
sentiment_chain = (
    ChatPromptTemplate.from_template("What is the sentiment of this: {input}")
    | llm
    | StrOutputParser()
)

parallel_chain = RunnableParallel({
    "summary": summary_chain,
    "sentiment": sentiment_chain
})
parallel_result = parallel_chain.invoke({"input": input_data})
print(parallel_result)

In [ ]:
from langchain_core.runnables import RunnableBranch
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

import os
from dotenv import load_dotenv

load_dotenv(override=True)

os.environ["DEEPSEEK_API_KEY"] = os.environ["SF_API_KEY"]
os.environ["DEEPSEEK_API_BASE"] = os.environ["SF_API_BASE"]

llm = ChatDeepSeek(
    model="deepseek-ai/DeepSeek-V3",
    temperature=0,
    max_retries=2,
    # other params...
)
input_data = "What is the capital of France?"
summary_chain = (
    ChatPromptTemplate.from_template("Summarize this: {input}")
    | llm
    | StrOutputParser()
)
sentiment_chain = (
    ChatPromptTemplate.from_template("What is the sentiment of this: {input}")
    | llm
    | StrOutputParser()
)

parallel_chain = RunnableParallel({
    "summary": summary_chain,
    "sentiment": sentiment_chain
})
parallel_result = parallel_chain.invoke({"input": input_data})
print(parallel_result)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_deepseek import ChatDeepSeek


llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_retries=2,
    # other params...
)
messages = [
    SystemMessage("Translate the following from English into Chinese."),
    HumanMessage("hi!"),
]

print(llm.invoke(messages).content)

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_retries=2,
    # other params...
)

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

chain = prompt_template | llm | StrOutputParser()
# print(chain)
print(chain.invoke({"language": "Chinese", "text": "hi!"}))


In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_retries=2,
    # other params...
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个{role}"),
    ("human", "根据用户描述生成{format}：{input}")
])
chain = prompt | llm | StrOutputParser()
print(chain.invoke({
    "role": "诗人", 
    "format": "五言绝句",
    "input": "秋天的夜晚"
}))


In [ ]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool

import os
from dotenv import load_dotenv

load_dotenv(override=True)

os.environ["DEEPSEEK_API_KEY"] = os.environ["SF_API_KEY"]
os.environ["DEEPSEEK_API_BASE"] = os.environ["SF_API_BASE"]

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm = ChatDeepSeek(
    model="deepseek-ai/DeepSeek-V3",
    temperature=0,
    max_retries=2,
    # other params...
)

llm_with_tools = llm.bind_tools(tools)

messages = [HumanMessage("what is 11 add 49?")]

ai_msg = llm_with_tools.invoke(messages)
# print(ai_msg)
messages.append(ai_msg)

tool_calls = ai_msg.tool_calls
# print(tool_calls)

for tool_call in tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    # print(tool_msg)
    messages.append(tool_msg)

# print(messages)
ai_msg = llm_with_tools.invoke(messages) 
print(ai_msg)

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

import os
from dotenv import load_dotenv

load_dotenv(override=True)

os.environ["DEEPSEEK_API_KEY"] = os.environ["SF_API_KEY"]
os.environ["DEEPSEEK_API_BASE"] = os.environ["SF_API_BASE"]

llm = ChatDeepSeek(
    model="deepseek-ai/DeepSeek-V3",
    temperature=0,
    max_retries=2,
    # other params...
)

# 定义子链
tech_prompt = ChatPromptTemplate.from_template(
    "作为科技作者，用专业术语解释：{concept}"
)
sports_prompt = ChatPromptTemplate.from_template(
    "作为体育解说员，用生动语言描述：{concept}"
)
general_prompt = ChatPromptTemplate.from_template(
    "用通俗语言解释：{concept}"
)

tech_chain = tech_prompt | llm | StrOutputParser()
sports_chain = sports_prompt | llm | StrOutputParser()
general_chain = general_prompt | llm | StrOutputParser()

# 构建分支逻辑
branch_chain = RunnableBranch(
    (lambda x: x["topic"] == "科技", tech_chain),
    (lambda x: x["topic"] == "体育", sports_chain),
    general_chain
)

# 完整执行链
# full_chain = RunnableParallel({  # 并行处理输入
#     "concept": lambda x: x["concept"],
#     "topic": lambda x: x["topic"]
# }) | branch_chain
full_chain = branch_chain

# 测试不同分支
print(full_chain.invoke({
    "concept": "神经网络", 
    "topic": "科技"
}))
# 输出包含"神经元"、"反向传播"等术语

print(full_chain.invoke({
    "concept": "越位", 
    "topic": "体育"
}))
# 输出包含"足球比赛"、"裁判判罚"等描述


In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel


# 导入环境变量
import os
from dotenv import load_dotenv

load_dotenv(override=True)

os.environ["DEEPSEEK_API_KEY"] = os.environ["SF_API_KEY"]
os.environ["DEEPSEEK_API_BASE"] = os.environ["SF_API_BASE"]

llm = ChatDeepSeek(
    model="deepseek-ai/DeepSeek-V3",
    temperature=0,
    max_retries=2,
    # other params...
)

# 定义子链
prompt = ChatPromptTemplate.from_template("总结文本：{text}")
# debug_prompt = prompt | (lambda x: print("提示内容:\n", x.to_messages()) or x)
summary_chain = (
    prompt | (lambda x: print("提示内容:\n", x.to_messages()) or x)
    | llm 
    | StrOutputParser()
)

sentiment_chain = (
    ChatPromptTemplate.from_template("分析文本情感倾向：{text}")
    | llm 
    | StrOutputParser()
)

# 并行处理链
parallel_chain = RunnableParallel({
    "summary": summary_chain,
    "sentiment": sentiment_chain
})

# 执行
input_data = {"text": "这款手机拍照效果出色，但电池续航较差"}
result = parallel_chain.invoke(input_data)
print(result)

#print(f"总结：{result['summary']}")
# 输出：总结：该手机拍照功能优秀，但电池续航能力不足
#print(f"情感：{result['sentiment']}")
# 输出：情感：正面与负面评价并存


提示内容:
 [HumanMessage(content='总结文本：这款手机拍照效果出色，但电池续航较差', additional_kwargs={}, response_metadata={})]
{'summary': '## 手机优缺点总结：\n\n**优点：**\n\n* 拍照效果出色\n\n**缺点：**\n\n* 电池续航较差', 'sentiment': '这段文本的情感倾向是**中性偏负面**。具体分析如下：\n\n1. **正面情感**：提到“拍照效果出色”，这是对手机的一个积极评价，表明用户对手机的拍照功能非常满意。\n2. **负面情感**：提到“电池续航较差”，这是对手机的一个消极评价，表明用户对电池续航能力不满意。\n\n虽然文本中同时包含了正面和负面的评价，但由于电池续航是用户日常使用中非常重要的一个方面，因此整体情感倾向偏向负面。'}


In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
from langchain_core.globals import set_debug

set_debug(True)
# 导入环境变量
import os
from dotenv import load_dotenv

load_dotenv(override=True)

os.environ["DEEPSEEK_API_KEY"] = os.environ["SF_API_KEY"]
os.environ["DEEPSEEK_API_BASE"] = os.environ["SF_API_BASE"]


llm = ChatDeepSeek(
    model="deepseek-ai/DeepSeek-V3",
    temperature=0,
    max_retries=2,
    # other params...
)

# 定义子链
prompt = ChatPromptTemplate.from_template("总结文本：{text}")
# debug_prompt = prompt | (lambda x: print("提示内容:\n", x.to_messages()) or x)
summary_chain = (
    prompt | (lambda x: print("提示内容:\n", x.to_messages()) or x)
    | llm 
    | StrOutputParser()
)

sentiment_chain = (
    ChatPromptTemplate.from_template("分析文本情感倾向：{text}")
    | llm 
    | StrOutputParser()
)

# 并行处理链
parallel_chain = RunnableParallel({
    "summary": summary_chain,
    "sentiment": sentiment_chain
})

# 执行
input_data = {"text": "这款手机拍照效果出色，但电池续航较差"}
result = parallel_chain.invoke(input_data)
# print(result)

#print(f"总结：{result['summary']}")
# 输出：总结：该手机拍照功能优秀，但电池续航能力不足
#print(f"情感：{result['sentiment']}")
# 输出：情感：正面与负面评价并存

